In [24]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "neuronal-sequence-test"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

# User libraries
from src.lib.qt_wrapper import gui_fnames, gui_fpath
from src.lib.data_db import BehaviouralNeuronalDatabase
import src.lib.plots_lib as plots_lib

%load_ext autoreload
%autoreload 2

Appended root directory /media/aleksejs/DataHDD/work/codes/comp-neuro/analysis-neuro/neuronal-sequence-test
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TODO

1. Preprocessing : Apply background subtraction
2. Alignment : Ensure intervals actually match. Ask Maria for visual test of alignment
3. Apply metrics - 1 data point per trial - pool trials - maintenance vs all
    * 1D semistatic: Mean, Variance, 1D Entropy
    * ND semistatic: Avg Correlation, ND Entropy
    * 1D dynamic: Predictive Info, AR(1) efficiency
    * ND dynamic: Predictive Info, AR(1) efficiency, Orderability coeff
    
4. Test Hypotheses:
    1. Maintenance predicts Correct/Incorrect vs other phases
    2. Maintenance predicts L/R better vs other phases
    3. Validation: Prev trial C/I or L/R predicts that of next trial
    4. Predictive aspect:
        * Semi-static maintenance activity is good predictor, fine temporal details irrelevant
        * Semi-static maintenance activity is not good, fine temporal details improve prediction

In [2]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
params['root_path_data']  = gui_fpath("Path to data files", "./")
#params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/maria'

In [3]:
dataDB = BehaviouralNeuronalDatabase(params)

In [4]:
dataDB.read_neuro_files()

IntProgress(value=0, description='Read Neuro Data:', max=14)

In [5]:
dataDB.read_behavior_files()

IntProgress(value=0, description='Read Neuro Data:', max=14)

Reading /media/aleksejs/DataHDD/work/data/maria/m060/Behavior_m060_20190522.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m060/Behavior_m060_20190524.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m060/Behavior_m060_20190527.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m060/Behavior_m060_20190606.mat
No trials found for Trial_LWhole_Mistake skipping
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191128.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191204.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191205.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191206.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m065/Behavior_m065_20191209.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m068/Behavior_m068_20200130.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m068/Behavior_m068_20200131.mat
Reading /media/aleksejs/DataHDD/work/data/maria/m068/Behavior_m068_202

In [88]:
plots_lib.table_test_metric_phase_vs_all(dataDB, "Maintenance", "mean", np.mean)

No behaviour found for {'performance': 'Mistake', 'direction': 'L', 'mousekey': 'm060_20190606'} ; skipping
No behaviour found for {'performance': 'Mistake', 'direction': 'L', 'mousekey': 'm060_20190606'} ; skipping
No behaviour found for {'performance': 'Mistake', 'direction': 'L', 'mousekey': 'm060_20190606'} ; skipping
No behaviour found for {'performance': 'Mistake', 'direction': 'L', 'mousekey': 'm060_20190606'} ; skipping
No behaviour found for {'direction': 'L', 'performance': 'Mistake', 'mousekey': 'm060_20190606'} ; skipping
No behaviour found for {'direction': 'L', 'performance': 'Mistake', 'mousekey': 'm060_20190606'} ; skipping
No behaviour found for {'direction': 'L', 'performance': 'Mistake', 'mousekey': 'm060_20190606'} ; skipping
No behaviour found for {'direction': 'L', 'performance': 'Mistake', 'mousekey': 'm060_20190606'} ; skipping


,performance,direction,datatype,mean(Maintenance),mean(WholeTrial),pVal_log10,nTrial
0,Correct,L,raw,-2.236264,-0.697467,-1.373570,256
1,Correct,L,high,0.835604,0.916089,-5.997608,256
2,Correct,R,raw,-0.875659,0.112680,-0.952086,233
3,Correct,R,high,0.861503,0.947460,-4.500450,233
4,Correct,All,raw,-1.587959,-0.311446,-1.772974,489
5,Correct,All,high,0.847945,0.931037,-9.579105,489
6,Mistake,L,raw,-2.753138,0.177327,-2.065900,71
7,Mistake,L,high,0.745697,0.924109,-4.518954,71
8,Mistake,R,raw,-1.765620,-0.053517,-0.990437,69
9,Mistake,R,high,0.836264,0.935980,-2.185633,69


In [115]:
queryThis = {"mousename" : "m060", "direction" : "L", "datatype" : "raw"}

plots_lib.table_binary_classification(dataDB, "Maintenance", queryThis, "performance") 

{'mousename': 'm060', 'direction': 'L', 'datatype': 'raw', 'performance': 'Correct'} (74, 88)
No behaviour found for {'direction': 'L', 'performance': 'Mistake', 'mousekey': 'm060_20190606'} ; skipping
{'mousename': 'm060', 'direction': 'L', 'datatype': 'raw', 'performance': 'Mistake'} (5, 88)


{'Accuracy_Test : ': 0.76625,
 'Accuracy_Train': 1.0,
 'p-value': 0.9225945717617944}